In [1]:
from Postprocess_functions import *

Welcome to JupyROOT 6.26/10


In [2]:
import gmsh
import sys
import os
from os.path import exists
import subprocess
import numpy as np
import time
import ROOT
from os import listdir
from os.path import isfile, join
import os.path
from os import path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import argparse
from ROOT import TFile, TTree
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev,splprep
from scipy.interpolate import BSpline
import math
def Initialization_Mult(args):
    print('Initializations')
    gen=[]
    gen_file=[]
    NSGA_files=[]
    root_folder=[]
    pictures_folder=[]
    pictures_folder_pareto=[]
    pictures_folder_filter=[]
    pictures_folder_filter_shape=[]
    
    c=0
    for fol in args.folder:
        root_folder.append(fol)
        gen.append(args.generation[c])
        print(gen)
        folder_path='ROOT/'+root_folder[c] # add path to ROOT/
        rf=os.listdir(folder_path)		# get all files in the folder
        NSGA_filenames = find_strings_containing_substring(rf, 'NSGAII_') # Get all root files
        print(rf)
        NSGA_filenames=sort_strings_by_int(NSGA_filenames)
        prefix = folder_path+'/'
        NSGA_files.append([prefix+s for s in NSGA_filenames])
        print(str(gen[c]))
        gen_file.append(folder_path+'/'+find_strings_containing_substring(rf, str(gen[c]))[0]) # Get last generation file or given gen.
        print('pictures/ROOT/'+root_folder[c]+'/Shape/')
        pictures_folder.append('pictures/ROOT/'+root_folder[c]+'/Shape/')
        print(pictures_folder)
        pictures_folder_pareto.append('pictures/ROOT/'+root_folder[c]+'/Pareto/')
        pictures_folder_filter.append('pictures/ROOT/'+root_folder[c]+'/Filter/')
        pictures_folder_filter_shape.append('pictures/ROOT/'+root_folder[c]+'/Filter/Shape_gen'+str(gen[c])+'/')

        create_folder(pictures_folder[c])  # Create required folders to store pictures
        create_folder(pictures_folder_pareto[c])  # Create required folders to store pictures
        create_folder(pictures_folder_filter[c])  # Create required folders to store pictures
        create_folder(pictures_folder_filter_shape[c])  # Create required folders to store pictures
        c+=1
        print('### Initializations END')
    return gen, gen_file, NSGA_files, root_folder, pictures_folder, pictures_folder_pareto, pictures_folder_filter, pictures_folder_filter_shape


In [23]:
def NSGAIIWrite_test1(features,  ROOTName, Generation,SubName ='SubFileGen',
                     JobName='JobActionNSGATestGmshPost.sh', 
                     Singularity='+SingularityImage="/storage/af/user/greales/SingDir/sandG4Gmsh"', 
                     SiPMS=False, LYSOL=False):
            print("### Writting Sub File:")
            nvar=len(features[0])
            nvar+=-1
            f = open("SubFiles/"+SubName+"_"+str(Generation)+".sub", "w")
            f.write("Universe = vanilla\n")
            f.write("executable = ../../../JobFiles/"+JobName+"\n")
            if(len(features)>1):
                #f.write('arguments ="-a Generation_'+str(self.Generation)+'_ -w $(indv) -v $(var)"\n')
                f.write('arguments ="-a '+ROOTName+'$(gen)_$(Item)_$(Cluster)_$(Process)_ -v $(var) -z $(nvar)')
                if(SiPMS):
                	f.write(' -t $(SiPM)')
                if(LYSOL):
                	f.write(' -l $(LYSOL)')	
                f.write('"\n')	
            
                f.write("Output  =/storage/af/user/greales/simG4/outputs/OutPost_$(gen)_$(vol)"+".out"+"\n")
               
# f.write("Output  ="+self.OutFolder+self.OutName+str(self.Generation)+"_1.out"+"\n")
            else:
                f.write('arguments ="-a Generation_$(gen)_$(indv)"\n')
                f.write("Output  ="+OutFolder+OutName+"_$(gen)_$(indv)"+".out"+"\n")
            f.write("Error   = /storage/af/user/greales/simG4/errors/error_job$(Cluster).out\n")
            f.write("Log     = /storage/af/user/greales/simG4/logs/log_job$(Cluster).out\n")
            f.write("requirements = Machine =!= LastRemoteHost\n")
            f.write("WHEN_TO_TRANSFER_OUTPUT = ON_EXIT_OR_EVICT\n")
            f.write('+JobQueue = "Short"\n')
            f.write("+MaxRuntime = 7000\n")
            f.write("+RunAsOwner = True\n")
            f.write("+InteractiveUser = True\n")
            f.write("+SingularityBindCVMFS = True\n")
            f.write(Singularity+"\n")
            f.write("x509userproxy = $ENV(X509_USER_PROXY)\n")
            f.write("RequestDisk = 2000000\n")
            f.write("RequestMemory = 2000\n")
            f.write("RequestCpus = 1\n")
            f.write("on_exit_remove = ((ExitBySignal == False) && (ExitCode == 0)) || (JobStatus=?=3)\n")
            f.write("on_exit_hold = (ExitBySignal == True) || (ExitCode != 0)\n")
            f.write("+PeriodicRemove = ((JobStatus =?= 2) && ((MemoryUsage =!= UNDEFINED && MemoryUsage > 2.5*RequestMemory)))\n")
            f.write("should_transfer_files = Yes\n")
            f.write("max_retries = 3\n")
            if(features==[]):
                f.write("Queue 1\n")
            else:
                checkoutnames=[]
                f.write("Queue var, gen, nvar, Item ")
                
                if(SiPMS):
                	f.write(', SiPM')
                if(LYSOL):
                	f.write(', LYSOL')
                f.write(' from (\n')	
                featinit=0
                if(LYSOL):
                	featinit=1
                
                cc=0
                for indvfeat in features:
                    print(indvfeat)
                    cmd="{"
                    for y in indvfeat[featinit:len(indvfeat)+1]:
                        cmd=cmd+"-"+str(y)
                    cmd=cmd+"}, "+str(Generation)+", "+str(nvar)+", "+str(cc)
                    if (SiPMS):
                        cmd=cmd+', '+str(indvfeat[featinit]*100)
                    if(LYSOL):
                        cmd=cmd+', '+str(indvfeat[0])
                    #cmd=cmd+'\n'
                    f.write(cmd+"\n")
                    cc+=1
                f.write(")\n")
            f.close()

In [10]:
# Fill the class with the required inputs:
class ArgValues:
    def __init__(self):
        self.folder = ["G11_objLC_Acte_NoYSym_SiMP_375x3_68Evts_Spline_23-02-12",
                       "G11_objLC_Acte_NoYSym_SiMPmatch3_375mm_68Evts_Spline_23-02-12",
                       "G11_objLC_Acte_NoYSym_SiMPmatch3_6mm_68Evts_Spline_23-02-12"]
        #self.folder = "G11_objLC_Acte_YSym_SiMPmatch6mm_68Evts_Spline_23-02-09"
        #self.folder ="G11_objLC_Acte_NoYSym_SiMPmatch3_6mm_68Evts_Spline_23-02-12"
        self.generation = [9,9,9]
        self.Ysym = [1,1,1]
        self.LYSOL = [1,1,1]
        self.save = 1
        self.Shape = 0
        self.ParetoFrontier = 0
        self.Filter = 1
        self.LaunchIndv = 0
        self.RootN = ['Pop_objLC_Acte_NoYSym_SiMP_375x3',
                     'Pop_objLC_Acte_NoYSym_SiMPmatch3_375mm',
                     'Pop_objLC_Acte_NoYSym_SiMPmatch3_6mm']
        self.Vol = '513'
        self.Subname = ["G11_objLC_Acte_NoYSym_SiMP_375x3_68Evts_Spline",
                       "G11_objLC_Acte_NoYSym_SiMPmatch3_375mm_Spline",
                       "G11_objLC_Acte_NoYSym_SiMPmatch3_6mm_Spline"]
        
args0=ArgValues()
#gen, gen_file, root_folder, pictures_folder, pictures_folder_pareto, pictures_folder_filter, pictures_folder_filter_shape = Initialization(args0)

In [11]:
gen, gen_file, NSGA_files, root_folder, pictures_folder, pictures_folder_pareto, pictures_folder_filter, pictures_folder_filter_shape= Initialization_Mult(args0)

In [14]:
num_indv,num_vars,obj1,obj2,vars_tuple,lstvars = GetRootVariables(gen_file[0])
len(vars_tuple[0])

The number of vars per indv is :11
The number indv is :200


11

In [24]:
for idx in [0,1,2]:
    num_indv,num_vars,obj1,obj2,vars_tuple,lstvars = GetRootVariables(gen_file[idx])
    NSGAIIWrite_test1(vars_tuple,  args0.RootN[idx], 16,SubName ='Sub'+args0.Subname[idx],
                     JobName='JobActionNSGATestGmshPopPostv2.sh', 
                     Singularity='+SingularityImage="/storage/af/user/greales/SingDir/sandG4Gmsh"', 
                     SiPMS=False, LYSOL=True)

The number of vars per indv is :11
The number indv is :200
### Writting Sub File:
[20.172084105129915, 0.4075216179623506, 0.3344196948821539, 0.26285054862208695, 0.26912437620848, 0.46329604923333584, 0.9200515298255662, 0.33804751408835626, 0.25, 0.25, 0.30428530951508204]
[19.791284809636597, 0.4964809371501183, 0.34633714583555997, 0.25, 0.25, 0.25, 0.904778636934447, 0.3897355359654376, 0.4923598416189069, 0.2562789301942228, 0.36810997193038114]
[21.478727425710254, 0.520986925504977, 0.25689175341289094, 0.25, 0.29030354684130993, 0.25, 1.072238100491664, 0.44190718928295214, 0.3168889379302212, 0.6282750352152647, 0.41613945478805286]
[20.60296160517071, 0.4698156530418364, 0.4681507456259843, 0.6744943843138096, 0.29199719932960605, 0.6444504764594507, 1.0600503258699623, 0.2919998144495001, 0.25, 0.3183772091505462, 0.33967164297515257]
[19.961918411103643, 0.7296434419916721, 0.4857715171459387, 0.30003785154432394, 0.28154262350789333, 0.31696017008788274, 0.97861631367750